In [1]:
import pandas as pd
import os

In [7]:
df = pd.read_csv('in/set4_homooligomers.csv', sep='\t')

In [8]:
df = df.drop_duplicates(subset='full_sequence', keep='first')

In [9]:
for key, value in df.iterrows():
    os.makedirs(f'calc/{key}', exist_ok=True)
    seq = value['full_sequence']
    pdb = value['pdb']
    
    with open(f'calc/{key}/{key}_monomer.csv', 'w') as f:
        f.write('id,sequence\n')
        f.write(f'{pdb},{seq}\n')
        
    with open(f'_SH/{key}.sh', 'w') as f:
        f.write('#!/bin/bash\n')
        f.write(f'cd ./../calc/{key}\n')
        f.write('source /opt/miniconda3/bin/activate cf_1.5\n')
        f.write(f'colabfold_batch {key}_monomer.csv . --num-models 5 --model-type alphafold2_multimer_v3 --num-recycle 5 --save-single-representations --save-pair-representations\n')
        
    

In [10]:
len(df)

246

In [11]:
with open(f'_SH/batch.sh', 'w') as f:
    f.write("""#!/bin/bash
#SBATCH -p gpu
#SBATCH -n 4
#SBATCH --gres=gpu:1
#SBATCH --mem=16GB
#SBATCH -J wio
#SBATCH --array 0-247

bash $SLURM_ARRAY_TASK_ID.sh

""")